# Data Loading

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import pickle
import warnings
warnings.filterwarnings('ignore')

*Kode tersebut mengimpor berbagai library penting yang digunakan untuk membangun sistem rekomendasi, seperti pandas dan numpy untuk manipulasi data, sklearn untuk menghitung kemiripan dan evaluasi model, serta scipy untuk mengelola data dalam bentuk matriks sparse agar lebih efisien. Selain itu, modul pickle digunakan untuk menyimpan dan memuat objek Python seperti model yang telah dilatih, dan warnings digunakan untuk menyembunyikan peringatan yang tidak penting agar output tetap bersih. Dengan konfigurasi ini, lingkungan kerja siap untuk membangun dan mengevaluasi sistem rekomendasi berbasis kemiripan antar data.*

In [2]:
print("\nLOADING PROCESSED DATA")
print("-" * 30)

try:
    # Load processed data from content-based notebook
    tourism_data = pd.read_csv('tourism_processed.csv')
    rating_data = pd.read_csv('rating_processed.csv')
    content_features = pd.read_csv('content_features.csv')

    # Load encoders and scalers
    with open('encoders.pkl', 'rb') as f:
        encoders = pickle.load(f)

    with open('scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    print("All processed data loaded successfully!")
    print(f"Tourism data: {tourism_data.shape}")
    print(f"Rating data: {rating_data.shape}")
    print(f"Content features: {content_features.shape}")

except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please run content_based_recommendation.ipynb first!")
    print("Make sure the 'data' folder exists with processed files.")
    exit()



LOADING PROCESSED DATA
------------------------------
All processed data loaded successfully!
Tourism data: (437, 10)
Rating data: (9921, 3)
Content features: (437, 11)


*Potongan kode tersebut digunakan untuk memuat data dan model yang telah diproses sebelumnya, yang dibutuhkan dalam sistem rekomendasi. Saat dijalankan, program akan mencetak pesan bahwa proses pemuatan data sedang berlangsung, lalu mencoba membaca tiga file CSV utama—yaitu data destinasi wisata (tourism_processed.csv), data rating pengguna (rating_processed.csv), dan data fitur konten (content_features.csv)—menggunakan pandas. Selain itu, file encoder (encoders.pkl) dan scaler (scaler.pkl) juga dimuat menggunakan modul pickle untuk mengembalikan objek preprocessing seperti label encoder atau normalizer yang telah dibuat sebelumnya. Jika semua file berhasil dimuat, maka akan ditampilkan informasi bentuk (jumlah baris dan kolom) dari masing-masing data sebagai konfirmasi. Namun, jika salah satu file tidak ditemukan, program akan menangkap error FileNotFoundError, menampilkan pesan kesalahan, memberi tahu pengguna bahwa mereka harus menjalankan notebook preprocessing terlebih dahulu, dan menghentikan eksekusi program menggunakan exit()*

# Data Preparation

In [3]:
print("\nDATA PREPARATION FOR COLLABORATIVE FILTERING")
print("-" * 30)

print("RATING DATA ANALYSIS:")
print(f"Total ratings: {len(rating_data)}")

if 'User_Id' in rating_data.columns:
    print(f"Unique users: {rating_data['User_Id'].nunique()}")
    print(f"Avg ratings per user: {len(rating_data) / rating_data['User_Id'].nunique():.2f}")

if 'Place_Id' in rating_data.columns:
    print(f"Unique places: {rating_data['Place_Id'].nunique()}")
    print(f"Avg ratings per place: {len(rating_data) / rating_data['Place_Id'].nunique():.2f}")

if 'Place_Ratings' in rating_data.columns:
    print(f"Rating range: {rating_data['Place_Ratings'].min()} - {rating_data['Place_Ratings'].max()}")

# Check for required columns
required_cols = ['User_Id', 'Place_Id', 'Place_Ratings']
missing_cols = [col for col in required_cols if col not in rating_data.columns]

if missing_cols:
    print(f"Missing required columns: {missing_cols}")
    print("Required columns for collaborative filtering: User_Id, Place_Id, Place_Ratings")
    exit()


DATA PREPARATION FOR COLLABORATIVE FILTERING
------------------------------
RATING DATA ANALYSIS:
Total ratings: 9921
Unique users: 300
Avg ratings per user: 33.07
Unique places: 437
Avg ratings per place: 22.70
Rating range: 1 - 5


*Potongan kode ini digunakan untuk menyiapkan dan menganalisis data sebelum digunakan dalam sistem collaborative filtering (rekomendasi berbasis interaksi pengguna). Pertama, program mencetak bahwa tahap persiapan data sedang dimulai. Kemudian dilakukan analisis awal terhadap data rating dengan menghitung total jumlah rating, jumlah pengguna unik (User_Id), rata-rata rating yang diberikan per pengguna, jumlah tempat unik (Place_Id), rata-rata rating yang diterima per tempat, serta rentang nilai rating (Place_Ratings). Analisis ini membantu untuk memahami distribusi dan kepadatan interaksi antara pengguna dan tempat. Setelah itu, program memeriksa apakah kolom-kolom penting yang dibutuhkan untuk collaborative filtering—yaitu User_Id, Place_Id, dan Place_Ratings—ada di dalam data. Jika salah satu dari kolom tersebut tidak ada, program mencetak peringatan bahwa kolom penting hilang dan menghentikan proses menggunakan exit(), karena tanpa ketiga kolom tersebut model collaborative filtering tidak bisa dibangun.*

In [4]:
print("\nCREATING USER-ITEM MATRIX")
print("-" * 30)

# Create user-item rating matrix
print("Creating user-item matrix...")

user_item_matrix = rating_data.pivot_table(
    index='User_Id',
    columns='Place_Id',
    values='Place_Ratings',
    fill_value=0
)

print(f"User-item matrix created!")
print(f"Shape: {user_item_matrix.shape}")
print(f"Sparsity: {(user_item_matrix == 0).sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100:.2f}%")

# Convert to sparse matrix for efficiency
user_item_sparse = csr_matrix(user_item_matrix.values)
print(f"Converted to sparse matrix for efficiency")


CREATING USER-ITEM MATRIX
------------------------------
Creating user-item matrix...
User-item matrix created!
Shape: (300, 437)
Sparsity: 92.68%
Converted to sparse matrix for efficiency


*Kode ini membuat user-item matrix dari data rating, di mana baris merepresentasikan pengguna dan kolom merepresentasikan tempat wisata, dengan nilai berupa rating yang diberikan. Nilai kosong diisi dengan nol. Setelah matriks dibuat, ditampilkan ukuran (shape) dan tingkat kelangkaannya (sparsity), yang menunjukkan seberapa banyak data yang bernilai nol. Untuk efisiensi komputasi, matriks kemudian dikonversi ke bentuk sparse matrix menggunakan csr_matrix.*

In [5]:
print("\nUSER-BASED COLLABORATIVE FILTERING")
print("-" * 30)

print("Computing user similarity matrix...")

# Calculate user similarity using cosine similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

print(f"User similarity matrix created: {user_similarity_df.shape}")

def get_user_based_recommendations(user_id, n_recommendations=5, min_similarity=0.1):
    """
    Get recommendations using user-based collaborative filtering
    """
    try:
        if user_id not in user_similarity_df.index:
            print(f"User {user_id} not found!")
            return None

        # Get similar users
        similar_users = user_similarity_df[user_id].sort_values(ascending=False)
        similar_users = similar_users[similar_users > min_similarity]
        similar_users = similar_users.drop(user_id)  # Remove the user itself

        if len(similar_users) == 0:
            print(f"No similar users found for user {user_id}")
            return None

        print(f"Found {len(similar_users)} similar users")

        # Get places rated by the target user
        user_ratings = user_item_matrix.loc[user_id]
        unrated_places = user_ratings[user_ratings == 0].index

        if len(unrated_places) == 0:
            print(f"User {user_id} has rated all places!")
            return None

        # Calculate predicted ratings for unrated places
        place_scores = {}

        for place_id in unrated_places:
            weighted_sum = 0
            similarity_sum = 0

            for similar_user, similarity in similar_users.head(10).items():  # Top 10 similar users
                if user_item_matrix.loc[similar_user, place_id] > 0:
                    weighted_sum += similarity * user_item_matrix.loc[similar_user, place_id]
                    similarity_sum += similarity

            if similarity_sum > 0:
                place_scores[place_id] = weighted_sum / similarity_sum

        # Sort and get top recommendations
        sorted_places = sorted(place_scores.items(), key=lambda x: x[1], reverse=True)
        top_recommendations = sorted_places[:n_recommendations]

        if not top_recommendations:
            print(f"No recommendations could be generated for user {user_id}")
            return None

        # Create recommendations dataframe
        rec_data = []
        for place_id, predicted_rating in top_recommendations:
            place_info = {'Place_Id': place_id, 'Predicted_Rating': predicted_rating}

            # Add place details if available
            if 'Place_Id' in tourism_data.columns:
                place_details = tourism_data[tourism_data['Place_Id'] == place_id]
                if not place_details.empty:
                    if 'Place_Name' in place_details.columns:
                        place_info['Place_Name'] = place_details.iloc[0]['Place_Name']
                    if 'Category' in place_details.columns:
                        place_info['Category'] = place_details.iloc[0]['Category']
                    if 'City' in place_details.columns:
                        place_info['City'] = place_details.iloc[0]['City']

            rec_data.append(place_info)

        recommendations_df = pd.DataFrame(rec_data)
        return recommendations_df

    except Exception as e:
        print(f"Error in user-based recommendations: {e}")
        return None


USER-BASED COLLABORATIVE FILTERING
------------------------------
Computing user similarity matrix...
User similarity matrix created: (300, 300)


*Potongan kode ini menerapkan metode user-based collaborative filtering untuk memberikan rekomendasi wisata berdasarkan kemiripan antar pengguna. Pertama, program menghitung user similarity matrix menggunakan cosine similarity dari matriks user-item. Lalu, fungsi get_user_based_recommendations dibuat untuk memberikan rekomendasi bagi pengguna tertentu. Fungsi ini akan mencari pengguna yang mirip, mengecek tempat wisata yang belum dirating oleh pengguna target, lalu menghitung prediksi rating untuk tempat-tempat tersebut berdasarkan rating dari pengguna serupa. Hasil akhirnya adalah daftar rekomendasi tempat wisata dengan rating prediksi tertinggi, lengkap dengan detail nama tempat, kategori, dan kota jika tersedia.*

In [6]:
print("\nITEM-BASED COLLABORATIVE FILTERING")
print("-" * 30)

print("Computing item similarity matrix...")

# Calculate item similarity using cosine similarity
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

print(f"Item similarity matrix created: {item_similarity_df.shape}")

def get_item_based_recommendations(user_id, n_recommendations=5, min_similarity=0.1):
    """
    Get recommendations using item-based collaborative filtering
    """
    try:
        if user_id not in user_item_matrix.index:
            print(f"User {user_id} not found!")
            return None

        # Get user's ratings
        user_ratings = user_item_matrix.loc[user_id]
        rated_places = user_ratings[user_ratings > 0]
        unrated_places = user_ratings[user_ratings == 0].index

        if len(rated_places) == 0:
            print(f"User {user_id} has no ratings!")
            return None

        if len(unrated_places) == 0:
            print(f"User {user_id} has rated all places!")
            return None

        # Calculate predicted ratings for unrated places
        place_scores = {}

        for place_id in unrated_places:
            weighted_sum = 0
            similarity_sum = 0

            # Find similar items that user has rated
            similar_items = item_similarity_df[place_id].sort_values(ascending=False)
            similar_items = similar_items[similar_items > min_similarity]

            for similar_place, similarity in similar_items.items():
                if similar_place in rated_places.index and rated_places[similar_place] > 0:
                    weighted_sum += similarity * rated_places[similar_place]
                    similarity_sum += similarity

            if similarity_sum > 0:
                place_scores[place_id] = weighted_sum / similarity_sum

        # Sort and get top recommendations
        sorted_places = sorted(place_scores.items(), key=lambda x: x[1], reverse=True)
        top_recommendations = sorted_places[:n_recommendations]

        if not top_recommendations:
            print(f"No recommendations could be generated for user {user_id}")
            return None

        # Create recommendations dataframe
        rec_data = []
        for place_id, predicted_rating in top_recommendations:
            place_info = {'Place_Id': place_id, 'Predicted_Rating': predicted_rating}

            # Add place details if available
            if 'Place_Id' in tourism_data.columns:
                place_details = tourism_data[tourism_data['Place_Id'] == place_id]
                if not place_details.empty:
                    if 'Place_Name' in place_details.columns:
                        place_info['Place_Name'] = place_details.iloc[0]['Place_Name']
                    if 'Category' in place_details.columns:
                        place_info['Category'] = place_details.iloc[0]['Category']
                    if 'City' in place_details.columns:
                        place_info['City'] = place_details.iloc[0]['City']

            rec_data.append(place_info)

        recommendations_df = pd.DataFrame(rec_data)
        return recommendations_df

    except Exception as e:
        print(f"Error in item-based recommendations: {e}")
        return None


ITEM-BASED COLLABORATIVE FILTERING
------------------------------
Computing item similarity matrix...
Item similarity matrix created: (437, 437)


*Kode ini menerapkan metode item-based collaborative filtering untuk memberikan rekomendasi tempat wisata berdasarkan kemiripan antar tempat. Proses dimulai dengan menghitung item similarity matrix menggunakan cosine similarity antar kolom (tempat) dalam matriks user-item. Fungsi get_item_based_recommendations kemudian digunakan untuk memprediksi rating tempat yang belum dikunjungi oleh pengguna berdasarkan rating pengguna terhadap tempat lain yang mirip. Prediksi dilakukan dengan menghitung rata-rata tertimbang dari rating pengguna terhadap tempat yang mirip, lalu diurutkan dan diambil beberapa rekomendasi terbaik. Hasil rekomendasi dilengkapi dengan informasi nama tempat, kategori, dan kota jika tersedia.*

In [7]:
print("\nMODEL EVALUATION")
print("-" * 30)

from sklearn.metrics import mean_squared_error
import numpy as np

def evaluate_user_item_based_cf(user_similarity_matrix, item_similarity_matrix, test_size=0.2, random_state=42):
    """
    Evaluate User-Based and Item-Based Collaborative Filtering models
    """
    try:
        print("Evaluating User-Based and Item-Based CF models...")

        np.random.seed(random_state)

        # Get all non-zero ratings
        non_zero_ratings = []
        for user_id in user_item_matrix.index:
            for place_id in user_item_matrix.columns:
                rating = user_item_matrix.loc[user_id, place_id]
                if rating > 0:
                    non_zero_ratings.append((user_id, place_id, rating))

        # Shuffle and split
        np.random.shuffle(non_zero_ratings)
        split_idx = int(len(non_zero_ratings) * (1 - test_size))
        train_ratings = non_zero_ratings[:split_idx]
        test_ratings = non_zero_ratings[split_idx:]

        print(f"Train ratings: {len(train_ratings)}")
        print(f"Test ratings: {len(test_ratings)}")

        # Create train matrix
        train_matrix = user_item_matrix.copy()
        for user_id, place_id, _ in test_ratings:
            train_matrix.loc[user_id, place_id] = 0

        def predict_user_based(user_id, place_id):
            if place_id not in train_matrix.columns or user_id not in train_matrix.index:
                return np.nan

            sim_scores = user_similarity_matrix[user_id]
            ratings = train_matrix[place_id]

            mask = (ratings > 0)
            if not mask.any():
                return np.nan

            weighted_sum = np.dot(sim_scores[mask], ratings[mask])
            sum_weights = np.sum(np.abs(sim_scores[mask]))

            return weighted_sum / sum_weights if sum_weights != 0 else np.nan

        def predict_item_based(user_id, place_id):
            if place_id not in train_matrix.columns or user_id not in train_matrix.index:
                return np.nan

            sim_scores = item_similarity_matrix[place_id]
            ratings = train_matrix.loc[user_id]

            mask = (ratings > 0)
            if not mask.any():
                return np.nan

            weighted_sum = np.dot(sim_scores[mask], ratings[mask])
            sum_weights = np.sum(np.abs(sim_scores[mask]))

            return weighted_sum / sum_weights if sum_weights != 0 else np.nan

        # Evaluate User-Based CF
        user_predictions, user_actuals = [], []
        for user_id, place_id, actual in test_ratings:
            pred = predict_user_based(user_id, place_id)
            if not np.isnan(pred):
                user_predictions.append(pred)
                user_actuals.append(actual)

        # Evaluate Item-Based CF
        item_predictions, item_actuals = [], []
        for user_id, place_id, actual in test_ratings:
            pred = predict_item_based(user_id, place_id)
            if not np.isnan(pred):
                item_predictions.append(pred)
                item_actuals.append(actual)

        def calc_metrics(preds, actuals):
            rmse = np.sqrt(mean_squared_error(actuals, preds))
            mae = np.mean(np.abs(np.array(actuals) - np.array(preds)))
            return rmse, mae

        if user_predictions:
            user_rmse, user_mae = calc_metrics(user_predictions, user_actuals)
            print(f"\nUser-Based CF Evaluation:")
            print(f"RMSE: {user_rmse:.4f}")
            print(f"MAE: {user_mae:.4f}")
        else:
            user_rmse = user_mae = None
            print("\nUser-Based CF could not be evaluated.")

        if item_predictions:
            item_rmse, item_mae = calc_metrics(item_predictions, item_actuals)
            print(f"\nItem-Based CF Evaluation:")
            print(f"RMSE: {item_rmse:.4f}")
            print(f"MAE: {item_mae:.4f}")
        else:
            item_rmse = item_mae = None
            print("\nItem-Based CF could not be evaluated.")

        return {
            'user_based': {
                'rmse': user_rmse,
                'mae': user_mae,
                'samples': len(user_predictions)
            },
            'item_based': {
                'rmse': item_rmse,
                'mae': item_mae,
                'samples': len(item_predictions)
            }
        }

    except Exception as e:
        print(f"Error in model evaluation: {e}")
        return None


MODEL EVALUATION
------------------------------


1. Mengumpulkan rating yang bukan nol

non_zero_ratings menyimpan semua kombinasi (user_id, place_id, rating) yang memiliki rating lebih dari 0.

2. Split train dan test

* Mengacak seluruh data rating (np.random.shuffle)

* Split berdasarkan test_size=0.2 → 80% data untuk pelatihan, 20% untuk pengujian.

* Data test akan disimpan terpisah dan di-zero-out dari train_matrix.

3. Prediksi rating:

* User-based: Prediksi rating dari user ke tempat berdasarkan user lain yang mirip dan sudah memberikan rating pada tempat itu.

* Item-based: Prediksi rating dari user ke tempat berdasarkan tempat lain yang mirip dan sudah diberi rating oleh user tersebut.

Untuk keduanya:

* Menggunakan skor kesamaan (similarity)

* Mengalikan dengan rating yang tersedia

* Dibagi dengan jumlah bobot kesamaan (agar jadi rata-rata tertimbang)

4. Evaluasi dengan metrik:

* RMSE: Mengukur akar dari rata-rata kuadrat selisih prediksi dan aktual

* MAE: Mengukur rata-rata selisih absolut antara prediksi dan aktual

In [8]:
print("\nRECOMMENDATION DEMONSTRATION")
print("-" * 30)

# Get sample users for demonstration
sample_users = user_item_matrix.index[:1].tolist()
print(f"Demonstrating recommendations for sample users: {sample_users}")

for user_id in sample_users:
    print(f"\n USER {user_id} RECOMMENDATIONS:")
    print("=" * 40)

    # Show user's current ratings
    user_ratings = user_item_matrix.loc[user_id]
    rated_places = user_ratings[user_ratings > 0]

    print(f"User {user_id} has rated {len(rated_places)} places")
    if len(rated_places) > 0:
        print(f"   Average rating: {rated_places.mean():.2f}")
        print(f"   Rating range: {rated_places.min()} - {rated_places.max()}")

    print("\nGenerating recommendations...")

    # User-based recommendations
    print("\n1. USER-BASED COLLABORATIVE FILTERING:")
    user_based_recs = get_user_based_recommendations(user_id, n_recommendations=3)
    if user_based_recs is not None and not user_based_recs.empty:
        print(user_based_recs.to_string(index=False))
    else:
        print("   No recommendations available")

    # Item-based recommendations
    print("\n2. ITEM-BASED COLLABORATIVE FILTERING:")
    item_based_recs = get_item_based_recommendations(user_id, n_recommendations=3)
    if item_based_recs is not None and not item_based_recs.empty:
        print(item_based_recs.to_string(index=False))
    else:
        print("   No recommendations available")

    print("\n" + "=" * 80)



RECOMMENDATION DEMONSTRATION
------------------------------
Demonstrating recommendations for sample users: [1]

 USER 1 RECOMMENDATIONS:
User 1 has rated 29 places
   Average rating: 3.41
   Rating range: 2.0 - 5.0

Generating recommendations...

1. USER-BASED COLLABORATIVE FILTERING:
Found 68 similar users
 Place_Id  Predicted_Rating              Place_Name      Category    City
       37               5.0 Bumi Perkemahan Cibubur Taman Hiburan Jakarta
       49               5.0   Galeri Indonesia Kaya        Budaya Jakarta
       53               5.0          Rumah Sipitung        Budaya Jakarta

2. ITEM-BASED COLLABORATIVE FILTERING:
 Place_Id  Predicted_Rating               Place_Name      Category       City
      129          4.570901       Bukit Lintang Sewu Taman Hiburan Yogyakarta
      325          4.502747 Saung Angklung Mang Udjo        Budaya    Bandung
      432          4.502616              Taman Mundu Taman Hiburan   Surabaya



1. Memilih Sample User

*sample_users = user_item_matrix.index[:1].tolist()*

Memilih 1 pengguna pertama dari user_item_matrix untuk didemonstrasikan.

2. Menampilkan Informasi Rating

* Menampilkan jumlah tempat yang sudah dirating.

* Rata-rata rating dan rentang rating.

3. Membuat Rekomendasi

* Menggunakan dua fungsi yang diasumsikan sudah dibuat sebelumnya:

  * get_user_based_recommendations(user_id, n_recommendations=3)

  * get_item_based_recommendations(user_id, n_recommendations=3)

* Setiap fungsi akan mengembalikan rekomendasi dalam bentuk DataFrame yang ditampilkan dengan to_string(index=False).